In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import tqdm
import os
import numpy as np
import re
from scipy.stats import linregress

In [26]:
data_path = "./Data/312_Validatie_VKF_Predictiemodellen/rep312_"

# Prepatory work

In [4]:
execute_bool = False
if execute_bool:
    ids = []
    names = []
    for chunk in tqdm.tqdm(pd.read_csv(data_path+"mondata_validated.csv",sep=";", encoding = "ISO-8859-1",chunksize=10000000,iterator=True,low_memory=False),ascii=True):

        for itemid in chunk.VariableID.unique():
            path = "./Data/312_Validatie_VKF_Predictiemodellen/rep312_mondata_validated_vars/"+str(itemid)+".csv"

            if os.path.exists(path):
                chunk[chunk.VariableID==itemid].to_csv(path,mode='a',index=False, header=False)
            else:
                chunk[chunk.VariableID==itemid].to_csv(path,index=False)

            ids.append(itemid)
            names.append(chunk[chunk.VariableID==itemid].Var_Abbr.unique()[0])
            
    temp_df = pd.DataFrame({"id":ids,"Var_Abbr":names})
    temp_df = temp_df.drop_duplicates("id")
    temp_df.to_csv("./Data/312_Validatie_VKF_Predictiemodellen/rep312_mondata_validated_vars/dictionary.csv",index=False)
            

In [5]:
dict_df = pd.read_csv("./Data/312_Validatie_VKF_Predictiemodellen/rep312_mondata_validated_vars/dictionary.csv")

# Patient and time-independent variables aggregation

## patients + extra

In [4]:
cohort_df = pd.read_csv(data_path+"cohort.csv",sep=";", encoding = "ISO-8859-1")
cohort_df.loc[cohort_df.Length==-1,"Length"] = np.NaN
cohort_df

,ICUSessionID,HospAdmissionID,ICUAdmissionSequence,icu_adm_mins_s_hosp_adm,icu_los,AgeOnAdmission,Length,Weight,bmi,bsa,icuurg,surgstatus,PatICUAdmissionSurvival,ICUAdmissionTime
0,2,2,1,207,1118,41.15,172.0,75,25.35154,1.87943,Elective,None,non-survivor,2013-01-09 18:28:00.0
1,3,3,1,16718,1608,85.63,183.0,76,22.69402,1.97692,Elective,None,survivor,2013-01-13 07:48:00.0
2,4,4,1,14630,1185,73.86,175.0,70,22.85714,1.84814,Elective,None,survivor,2013-02-07 17:48:00.0
3,5,5,1,11535,853,16.94,156.0,60,24.65483,1.59256,Elective,None,survivor,2013-02-11 20:07:00.0
4,6,6,1,40117,3394,61.70,165.0,65,23.87511,1.71605,Urgent,Planned,survivor,2013-02-21 12:36:00.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25292,34289,31269,1,0,1160,62.27,165.0,70,25.71166,1.77096,Elective,None,survivor,2020-12-30 21:18:00.0
25293,34291,31271,1,132,7942,59.48,165.0,66,24.24242,1.72722,Urgent,None,survivor,2020-12-30 23:27:00.0
25294,34292,31272,1,185,1740,63.47,172.0,76,25.68956,1.89004,Elective,Emergency,survivor,2020-12-31 10:27:00.0
25295,34293,31273,1,292,5754,67.58,NaN,82,NaN,NaN,Elective,Emergency,survivor,2020-12-31 16:40:00.0


In [5]:
Extr_OpnameGegevens_df = pd.read_csv(data_path+"Extr_OpnameGegevens.csv",sep=";", encoding = "ISO-8859-1")
Extr_OpnameGegevens_df = Extr_OpnameGegevens_df[Extr_OpnameGegevens_df.variable=="sepsis opnam"]
Extr_OpnameGegevens_df["sepsis_bool_extra_opn"] = 0
Extr_OpnameGegevens_df.loc[Extr_OpnameGegevens_df.code==1,"sepsis_bool_extra_opn"] = 1
Extr_OpnameGegevens_df = Extr_OpnameGegevens_df.rename(columns={'mins_since_admission':'sepsis_extra_opn_measuredat'})
Extr_OpnameGegevens_df = Extr_OpnameGegevens_df[['ICUSessionID','sepsis_extra_opn_measuredat', 'sepsis_bool_extra_opn']]
Extr_OpnameGegevens_df = Extr_OpnameGegevens_df.drop_duplicates("ICUSessionID",keep="last")

In [6]:
AF_dataset_df = cohort_df.merge(Extr_OpnameGegevens_df,how="left",on="ICUSessionID")
AF_dataset_df = AF_dataset_df[['ICUSessionID', 'HospAdmissionID', 'ICUAdmissionSequence','AgeOnAdmission', 'ICUAdmissionTime','icu_los','icuurg','Length','Weight', 'bmi', 'bsa', 'sepsis_extra_opn_measuredat','sepsis_bool_extra_opn']]
AF_dataset_df.ICUAdmissionTime = pd.to_datetime(AF_dataset_df.ICUAdmissionTime)

AF_dataset_df

,ICUSessionID,HospAdmissionID,ICUAdmissionSequence,AgeOnAdmission,ICUAdmissionTime,icu_los,icuurg,Length,Weight,bmi,bsa,sepsis_extra_opn_measuredat,sepsis_bool_extra_opn
0,2,2,1,41.15,2013-01-09 18:28:00,1118,Elective,172.0,75,25.35154,1.87943,1080.0,1.0
1,3,3,1,85.63,2013-01-13 07:48:00,1608,Elective,183.0,76,22.69402,1.97692,494.0,0.0
2,4,4,1,73.86,2013-02-07 17:48:00,1185,Elective,175.0,70,22.85714,1.84814,327.0,0.0
3,5,5,1,16.94,2013-02-11 20:07:00,853,Elective,156.0,60,24.65483,1.59256,47.0,0.0
4,6,6,1,61.70,2013-02-21 12:36:00,3394,Urgent,165.0,65,23.87511,1.71605,32.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25292,34289,31269,1,62.27,2020-12-30 21:18:00,1160,Elective,165.0,70,25.71166,1.77096,49.0,0.0
25293,34291,31271,1,59.48,2020-12-30 23:27:00,7942,Urgent,165.0,66,24.24242,1.72722,49.0,0.0
25294,34292,31272,1,63.47,2020-12-31 10:27:00,1740,Elective,172.0,76,25.68956,1.89004,69.0,0.0
25295,34293,31273,1,67.58,2020-12-31 16:40:00,5754,Elective,NaN,82,NaN,NaN,41.0,0.0


## observrec

In [7]:
observrec_df = pd.read_csv(data_path+"observrec.csv",sep=";", encoding = "ISO-8859-1")
observrec_df

,icusessionid,mins_since_admission,variablename,stringvalue
0,5,487,VPK-Ritme,Sinusaal ritme
1,10,32,VPK-Ritme,Tachycardie
2,10,62,VPK-Ritme,Tachycardie
3,10,122,VPK-Ritme,Tachycardie
4,10,602,VPK-Ritme,Sinusaal ritme
...,...,...,...,...
629647,34294,942,VPK-Ritme,Tachycardie
629648,34294,972,VPK-Ritme,Tachycardie
629649,34294,1032,VPK-Ritme,Tachycardie
629650,34294,1092,VPK-Ritme,Tachycardie


In [8]:
af_icusession_id_df = observrec_df[observrec_df.stringvalue=="VKF"].drop_duplicates("icusessionid",keep="first")
af_icusession_id_df = af_icusession_id_df.rename(columns={"mins_since_admission":"AF_measuredat","icusessionid":"ICUSessionID"})
af_icusession_id_df["AF"] = 1
af_icusession_id_df = af_icusession_id_df[["ICUSessionID","AF_measuredat","AF"]]
af_icusession_id_df

AF_dataset_df = AF_dataset_df.merge(af_icusession_id_df,how="left",on="ICUSessionID")
AF_dataset_df.loc[AF_dataset_df.AF.isna(),"AF"]=0

AF_dataset_df.loc[AF_dataset_df.AF_measuredat.isna(),"AF_measuredat"]=0

AF_dataset_df

,ICUSessionID,HospAdmissionID,ICUAdmissionSequence,AgeOnAdmission,ICUAdmissionTime,icu_los,icuurg,Length,Weight,bmi,bsa,sepsis_extra_opn_measuredat,sepsis_bool_extra_opn,AF_measuredat,AF
0,2,2,1,41.15,2013-01-09 18:28:00,1118,Elective,172.0,75,25.35154,1.87943,1080.0,1.0,0.0,0.0
1,3,3,1,85.63,2013-01-13 07:48:00,1608,Elective,183.0,76,22.69402,1.97692,494.0,0.0,0.0,0.0
2,4,4,1,73.86,2013-02-07 17:48:00,1185,Elective,175.0,70,22.85714,1.84814,327.0,0.0,0.0,0.0
3,5,5,1,16.94,2013-02-11 20:07:00,853,Elective,156.0,60,24.65483,1.59256,47.0,0.0,0.0,0.0
4,6,6,1,61.70,2013-02-21 12:36:00,3394,Urgent,165.0,65,23.87511,1.71605,32.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25292,34289,31269,1,62.27,2020-12-30 21:18:00,1160,Elective,165.0,70,25.71166,1.77096,49.0,0.0,0.0,0.0
25293,34291,31271,1,59.48,2020-12-30 23:27:00,7942,Urgent,165.0,66,24.24242,1.72722,49.0,0.0,0.0,0.0
25294,34292,31272,1,63.47,2020-12-31 10:27:00,1740,Elective,172.0,76,25.68956,1.89004,69.0,0.0,0.0,0.0
25295,34293,31273,1,67.58,2020-12-31 16:40:00,5754,Elective,NaN,82,NaN,NaN,41.0,0.0,20.0,1.0


## Comorb

In [9]:
comorb_df = pd.read_csv(data_path+"comorb.csv", encoding = "ISO-8859-1",sep=";")
comorb_df.loc[comorb_df.Diagnose.isna(),"Diagnose"]=""
comorb_df

,ICUSessionID,CardialeVG,Diagnose
0,276,Geen (ikv score),hypertensieve thalamostriatale bloeding links ...
1,470,Oud AMI,OOHA > 45min REA op ritmestoornis
2,44,Geen (ikv score),severe sepsis obv speumonie
3,447,Geen (ikv score),Niertx (zonder nefrectomie)
4,402,Oud AMI,copd-opstoot (niet infectieus)
...,...,...,...
25289,34292,Geen (ikv score),stroke - thrombectomie en stent rechter carotis
25290,34294,Geen (ikv score),orbitale cellulitis
25291,34293,Geen (ikv score),Trombolyse
25292,34255,Geen (ikv score),"Neurologische achteruitgang, ischemisch CVA"


In [10]:
ids_sepsis = comorb_df[comorb_df.Diagnose.str.contains("sepsis",flags=re.IGNORECASE)].ICUSessionID.unique()
comorb_df[comorb_df.Diagnose.str.contains("sepsis",flags=re.IGNORECASE)].sort_values("ICUSessionID")

,ICUSessionID,CardialeVG,Diagnose
103,3,Geen (ikv score),obstructiebeeld met sepsis 13/3
13,35,Geen (ikv score),Sepsis ???
2,44,Geen (ikv score),severe sepsis obv speumonie
484,63,Oud AMI,begin sepsis (neutropenie)
175,64,Geen (ikv score),urosepsis
...,...,...,...
25113,34153,Geen (ikv score),kathetersepsis
25194,34164,Geen (ikv score),sepsis na gecompliceerde colonchirurgie
25181,34195,Vkf,sepsis en ANI
25243,34251,Geen (ikv score),"pneumococcensepsis, MOR"


In [11]:
AF_dataset_df["sepsis_bool_comorb_source"]=0
AF_dataset_df.loc[AF_dataset_df.ICUSessionID.isin(ids_sepsis),"sepsis_bool_comorb_source"]=1
AF_dataset_df

,ICUSessionID,HospAdmissionID,ICUAdmissionSequence,AgeOnAdmission,ICUAdmissionTime,icu_los,icuurg,Length,Weight,bmi,bsa,sepsis_extra_opn_measuredat,sepsis_bool_extra_opn,AF_measuredat,AF,sepsis_bool_comorb_source
0,2,2,1,41.15,2013-01-09 18:28:00,1118,Elective,172.0,75,25.35154,1.87943,1080.0,1.0,0.0,0.0,0
1,3,3,1,85.63,2013-01-13 07:48:00,1608,Elective,183.0,76,22.69402,1.97692,494.0,0.0,0.0,0.0,1
2,4,4,1,73.86,2013-02-07 17:48:00,1185,Elective,175.0,70,22.85714,1.84814,327.0,0.0,0.0,0.0,0
3,5,5,1,16.94,2013-02-11 20:07:00,853,Elective,156.0,60,24.65483,1.59256,47.0,0.0,0.0,0.0,0
4,6,6,1,61.70,2013-02-21 12:36:00,3394,Urgent,165.0,65,23.87511,1.71605,32.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25292,34289,31269,1,62.27,2020-12-30 21:18:00,1160,Elective,165.0,70,25.71166,1.77096,49.0,0.0,0.0,0.0,0
25293,34291,31271,1,59.48,2020-12-30 23:27:00,7942,Urgent,165.0,66,24.24242,1.72722,49.0,0.0,0.0,0.0,0
25294,34292,31272,1,63.47,2020-12-31 10:27:00,1740,Elective,172.0,76,25.68956,1.89004,69.0,0.0,0.0,0.0,0
25295,34293,31273,1,67.58,2020-12-31 16:40:00,5754,Elective,NaN,82,NaN,NaN,41.0,0.0,20.0,1.0,0


## apache

In [12]:
apache_2_df = pd.read_csv(data_path+"apache2.csv",sep=";", encoding = "ISO-8859-1")
apache_2_df

,ICUSessionID,apache2
0,2,40.0
1,3,12.0
2,4,15.0
3,5,8.0
4,6,10.0
...,...,...
25292,34289,19.0
25293,34291,21.0
25294,34292,11.0
25295,34293,18.0


In [13]:
AF_dataset_df = AF_dataset_df.merge(apache_2_df,how="left",on="ICUSessionID")
AF_dataset_df

,ICUSessionID,HospAdmissionID,ICUAdmissionSequence,AgeOnAdmission,ICUAdmissionTime,icu_los,icuurg,Length,Weight,bmi,bsa,sepsis_extra_opn_measuredat,sepsis_bool_extra_opn,AF_measuredat,AF,sepsis_bool_comorb_source,apache2
0,2,2,1,41.15,2013-01-09 18:28:00,1118,Elective,172.0,75,25.35154,1.87943,1080.0,1.0,0.0,0.0,0,40.0
1,3,3,1,85.63,2013-01-13 07:48:00,1608,Elective,183.0,76,22.69402,1.97692,494.0,0.0,0.0,0.0,1,12.0
2,4,4,1,73.86,2013-02-07 17:48:00,1185,Elective,175.0,70,22.85714,1.84814,327.0,0.0,0.0,0.0,0,15.0
3,5,5,1,16.94,2013-02-11 20:07:00,853,Elective,156.0,60,24.65483,1.59256,47.0,0.0,0.0,0.0,0,8.0
4,6,6,1,61.70,2013-02-21 12:36:00,3394,Urgent,165.0,65,23.87511,1.71605,32.0,0.0,0.0,0.0,0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25292,34289,31269,1,62.27,2020-12-30 21:18:00,1160,Elective,165.0,70,25.71166,1.77096,49.0,0.0,0.0,0.0,0,19.0
25293,34291,31271,1,59.48,2020-12-30 23:27:00,7942,Urgent,165.0,66,24.24242,1.72722,49.0,0.0,0.0,0.0,0,21.0
25294,34292,31272,1,63.47,2020-12-31 10:27:00,1740,Elective,172.0,76,25.68956,1.89004,69.0,0.0,0.0,0.0,0,11.0
25295,34293,31273,1,67.58,2020-12-31 16:40:00,5754,Elective,NaN,82,NaN,NaN,41.0,0.0,20.0,1.0,0,18.0


In [14]:
apache_4_df = pd.read_csv(data_path+"apache4.csv",sep=";", encoding = "ISO-8859-1")
apache_4_df

C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\IPython\core\interactiveshell.py:3397: DtypeWarning: Columns (7,15,17,22,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ICUSessionID,temp_value,temp_score,map_value,map_score,hr_value,hr_score,rr_value,rr_score,po2_value,...,aps3,aps4,aps5,ap4score,dx_code,dx_coeff,x,y,mor,sej
0,7,"35,8",2.0,103.0,4.0,102.0,1.0,43.0,11.0,NaN,...,0,0,0,23,DXOM_BACCAP,-0.04336591435339050,NaN,NaN,NaN,NaN
1,8,"37,3",0.0,78.0,6.0,105.0,1.0,26.0,6.0,"176,2",...,0,0,0,23,DXOM_RESPASTM,-154.067.849.761.874.000,"-5,691685911","3,173126495","0,3","3,2"
2,21,"35,9",2.0,66.0,7.0,64.0,0.0,12.0,7.0,"28,2",...,250047,103823,216,108,NaN,NaN,NaN,"5,289696268",NaN,"5,3"
3,22,"34,90000153",8.0,69.0,7.0,88.0,0.0,11.0,8.0,"65,7",...,729,0,0,54,DXOS_THORCA,0.08693357600182190,"-2,145416155","4,242203553","10,5","4,2"
4,25,"35,79999924",2.0,55.0,15.0,100.0,1.0,18.0,0.0,56,...,405224,195112,4913,111,NaN,NaN,NaN,"7,413649072",NaN,"7,4"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25291,18892,"35,7",2.0,62.0,7.0,48.0,5.0,13.0,7.0,NaN,...,117649,35937,0,81,NaN,NaN,NaN,NaN,NaN,NaN
25292,57,"36,4",0.0,132.0,9.0,117.0,5.0,24.0,0.0,NaN,...,0,0,0,19,DXOS_GIOBST,-0.18900513198763800,NaN,NaN,NaN,NaN
25293,1929,"34,90000153",8.0,54.0,15.0,60.0,0.0,1.0,17.0,NaN,...,17576,1000,0,74,DXOS_THORCA,0.08693357600182190,NaN,NaN,NaN,NaN
25294,11939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,8,DXOM_NEUROTH,-0.17682869690988300,NaN,NaN,NaN,NaN


In [15]:
AF_dataset_df = AF_dataset_df.merge(apache_4_df[["ICUSessionID","ap4score"]],how="left",on="ICUSessionID")
AF_dataset_df

,ICUSessionID,HospAdmissionID,ICUAdmissionSequence,AgeOnAdmission,ICUAdmissionTime,icu_los,icuurg,Length,Weight,bmi,bsa,sepsis_extra_opn_measuredat,sepsis_bool_extra_opn,AF_measuredat,AF,sepsis_bool_comorb_source,apache2,ap4score
0,2,2,1,41.15,2013-01-09 18:28:00,1118,Elective,172.0,75,25.35154,1.87943,1080.0,1.0,0.0,0.0,0,40.0,162.0
1,3,3,1,85.63,2013-01-13 07:48:00,1608,Elective,183.0,76,22.69402,1.97692,494.0,0.0,0.0,0.0,1,12.0,62.0
2,4,4,1,73.86,2013-02-07 17:48:00,1185,Elective,175.0,70,22.85714,1.84814,327.0,0.0,0.0,0.0,0,15.0,74.0
3,5,5,1,16.94,2013-02-11 20:07:00,853,Elective,156.0,60,24.65483,1.59256,47.0,0.0,0.0,0.0,0,8.0,47.0
4,6,6,1,61.70,2013-02-21 12:36:00,3394,Urgent,165.0,65,23.87511,1.71605,32.0,0.0,0.0,0.0,0,10.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25292,34289,31269,1,62.27,2020-12-30 21:18:00,1160,Elective,165.0,70,25.71166,1.77096,49.0,0.0,0.0,0.0,0,19.0,58.0
25293,34291,31271,1,59.48,2020-12-30 23:27:00,7942,Urgent,165.0,66,24.24242,1.72722,49.0,0.0,0.0,0.0,0,21.0,136.0
25294,34292,31272,1,63.47,2020-12-31 10:27:00,1740,Elective,172.0,76,25.68956,1.89004,69.0,0.0,0.0,0.0,0,11.0,65.0
25295,34293,31273,1,67.58,2020-12-31 16:40:00,5754,Elective,NaN,82,NaN,NaN,41.0,0.0,20.0,1.0,0,18.0,55.0


## sec diagnosises

In [16]:
SecDiagnoses_df = pd.read_csv(data_path+"SecDiagnoses.csv",sep=";", encoding = "ISO-8859-1")
SecDiagnoses_df

,ICUSessionID,Code,Name
0,17948,2M-HYPONATR,hyponatriemie
1,18062,2M-ICUAW,ICU acquired weakness
2,18062,2M-VAP,ventilator associated pneumonie
3,18143,2M-ALI,acute lung injury ALI
4,18143,2M-PLEUREFFUSIE,pleuravocht
...,...,...,...
15587,34293,2HK-EMBOLPERIF,embolectomie perifere vaten
15588,34293,2M-COMASTUPOR,coma of stupor
15589,34294,2M-AKI,acuut nierfalen
15590,34294,2M-ALF,acuut leverfalen


# Preparing full dataset with preprocessing on time

## Preparing

In [17]:
one_half_hour_model = False
six_hour_model = True
non_biased_model = True #Match the NO AF measurement point distribution to the AF patients to avoid time-dependent treatment bias

if one_half_hour_model:
    time_shift = 1.5*60
    margin_time = 1.5*60
else:
    time_shift = 12*60
    margin_time = 0*60

if six_hour_model:
    time_shift = 6*60
    margin_time = 0*60
    
hours_to_first_AF = 12*60
total_window = (hours_to_first_AF+time_shift)#*60 #in minutes
total_window/60

to_hour_multiplier = 1

In [20]:
# AF_dataset_df_or = AF_dataset_df.copy(deep=True)

In [21]:
# AF_dataset_df = AF_dataset_df_or.copy(deep=True)

In [22]:
only_data_of_first_hours = False #Use only data of the first X hours of admission
include_AF_patients_without_AF_at_sample = True #Include moments in time of AF patients before the AF diagnosis in the dataset.

np.random.seed(42)

#exclusion, minimum 2014
AF_dataset_df = AF_dataset_df[AF_dataset_df.ICUAdmissionTime>pd.to_datetime("2014")]

AF_dataset_df = AF_dataset_df[(AF_dataset_df.icu_los>=total_window)].copy(deep=True)

AF_dataset_df = AF_dataset_df.sample(len(AF_dataset_df),random_state=42)
AF_dataset_df = AF_dataset_df[(AF_dataset_df.AF==0)|(AF_dataset_df.AF_measuredat>=to_hour_multiplier*total_window)]

if non_biased_model:
    AF_measuredat_sample_df = AF_dataset_df[(AF_dataset_df.AF==1)&(AF_dataset_df.AF_measuredat>(total_window))][["ICUSessionID","AF_measuredat"]].copy(deep=True)
    AF_dataset_df["date_corresponds_to_AF_admid"] = AF_dataset_df.ICUSessionID
    for ICUSessionID in AF_dataset_df[AF_dataset_df.AF==0].ICUSessionID.values:
        if len(AF_measuredat_sample_df) == 0:
            break
        else:
            if len(AF_measuredat_sample_df[AF_measuredat_sample_df.AF_measuredat<=((AF_dataset_df[AF_dataset_df.ICUSessionID==ICUSessionID]['icu_los'].values[0]))])>0:
                random_state_admission = np.random.RandomState(ICUSessionID)
                choice = random_state_admission.choice(AF_measuredat_sample_df[AF_measuredat_sample_df.AF_measuredat<=((AF_dataset_df[AF_dataset_df.ICUSessionID==ICUSessionID]['icu_los'].values[0]+0.1))]['ICUSessionID'].values)
                AF_dataset_df.loc[AF_dataset_df.ICUSessionID==ICUSessionID,"AF_measuredat"] = AF_measuredat_sample_df[AF_measuredat_sample_df.ICUSessionID==choice].AF_measuredat.values[0]
                AF_dataset_df.loc[AF_dataset_df.ICUSessionID==ICUSessionID,"date_corresponds_to_AF_admid"] = AF_measuredat_sample_df[AF_measuredat_sample_df.ICUSessionID==choice].ICUSessionID.values[0]
                AF_measuredat_sample_df = AF_measuredat_sample_df[AF_measuredat_sample_df.ICUSessionID!=choice]   
        # AF_dataset_df.loc[:,"AF_measuredat"] =  AF_dataset_df.apply(lambda row: np.random.randint(total_window,row['icu_los']+1) if (row['AF']==0) and (row['date_corresponds_to_AF_admid']==row['HospAdmissionID']) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.                     
    AF_dataset_df.loc[:,"AF_measuredat"] = AF_dataset_df.apply(lambda row: np.random.randint(total_window,row['icu_los']+1) if (row['AF']==0) and ((pd.isnull(row["AF_measuredat"])|(row["AF_measuredat"]==0))) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.                     
else:
    # AF_dataset_df.loc[:,"AF_measuredat"] =  AF_dataset_df.apply(lambda row: np.random.randint(total_window,row['icu_los']+1) if pd.isnull(row['AF_measuredat']) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.
    AF_dataset_df.loc[:,"AF_measuredat"] = AF_dataset_df.apply(lambda row: np.random.randint(total_window,row['icu_los']+1) if (row['AF']==0) and ((pd.isnull(row["AF_measuredat"])|(row["AF_measuredat"]==0))) else row['AF_measuredat'],axis=1).values #the no AF patients should have a timesample to "measure" AF.                     
    
# #AF patients
# first_AFs_pd_timed = AF_dataset_df[(AF_dataset_df.AF==1)&(AF_dataset_df.AF_measuredat>=total_window)].reset_index(drop=True)
# #indicate whether the patient is an AF patient or not, this is more for performance analysis of the model
# first_AFs_pd_timed["AF_orig"] = 1

# #No AF patients
# #no_AF_same_length = AF_dataset_df[(AF_dataset_df.AF==0)&(AF_dataset_df.icu_los>=total_window)].sample(len(first_AFs_pd_timed),random_state=1).reset_index(drop=True)
# no_AF_same_length = AF_dataset_df[(AF_dataset_df.AF==0)&(AF_dataset_df.icu_los>=total_window)].reset_index(drop=True)
# no_AF_same_length["AF_orig"] = 0

# temp_pd = no_AF_same_length


AF_dataset_df["AF_orig"]=AF_dataset_df.AF
#AF_admission_dataset = first_AFs_pd_timed.append(no_AF_same_length).reset_index()
AF_admission_dataset = AF_dataset_df[AF_dataset_df.AF_measuredat>=to_hour_multiplier*total_window]#pd.concat([first_AFs_pd_timed,temp_pd]).reset_index(drop=True) #AF_dataset

if only_data_of_first_hours:
    AF_admission_dataset["AF_measuredat"] = total_window #only take data of the first 48 hours
    
first_AFs_pd_timed = None
temp_pd = None

## labdata

In [151]:
labdata_df = pd.read_csv(data_path+"labdata.csv",sep=";", encoding = "ISO-8859-1")
labdata_df = labdata_df.rename(columns={"mins_since_icu_adm":"measuredat","VariableValue":"value"})
labdata_df

,AllLabDataID,ICUSessionID,VariableID,Var_Abbr,Unit,measuredat,value,admissionday7_7
0,42,2,10039,PLATELETS,10^3/mcL,313,27.000,0
1,41,2,10039,PLATELETS,10^3/mcL,695,18.000,0
2,9299324,2,10041,UREA_ALL,mg/dL,313,16.000,0
3,9299325,2,10041,UREA_ALL,mg/dL,695,16.000,0
4,55,2,10244,PH_ART_ALL,NaN,21,6.976,0
...,...,...,...,...,...,...,...,...
6370425,18487375,34294,11097,PCO2_PHVEN,mmHg,488,44.600,0
6370426,18487355,34294,11098,HCO3_PHVEN,mmol/L,488,19.200,0
6370427,18487345,34294,11099,BE_VENPH,mmol/L,488,-7.500,0
6370428,18487395,34294,11100,PO2_VENPH,mmHg,488,35.600,0


In [152]:
#NUMERIC ITEMS PREPROCESSING, this takes around 2 minutes
if non_biased_model:
    labdata_df_patients = (labdata_df[labdata_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='ICUSessionID')
else:
    labdata_df_patients = (labdata_df[labdata_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat"]],how='left',on='ICUSessionID')

labdata_df_patients["time_to_AF"]=(labdata_df_patients.AF_measuredat.values-margin_time) - labdata_df_patients.measuredat.values #add one margin_time to AF extra
labdata_df_patients = labdata_df_patients[(labdata_df_patients.time_to_AF > (time_shift-margin_time)) & (labdata_df_patients.time_to_AF <= (time_shift+hours_to_first_AF-margin_time) )]

for VariableID_loop in labdata_df.VariableID.unique():
    labdata_df_patients.VariableID = labdata_df_patients.VariableID.replace(VariableID_loop,labdata_df[labdata_df.VariableID==VariableID_loop].Var_Abbr.values[0])

labdata_df = None #RAM Optimization

labdata_df_patients_agg = labdata_df_patients[["ICUSessionID","VariableID","value"]].groupby(["ICUSessionID","VariableID"]).agg({'mean','min','max',pd.DataFrame.kurt}).reset_index()
labdata_df_patients_agg.VariableID = labdata_df_patients_agg.VariableID.astype(str)
labdata_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in labdata_df_patients_agg.columns.values]
labdata_df_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in labdata_df_patients_agg.columns.values]
labdata_df_patients_agg = labdata_df_patients_agg.pivot(index='ICUSessionID', columns='VariableID')
labdata_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in labdata_df_patients_agg.columns.values]

labdata_df_patients_slope = labdata_df_patients[["ICUSessionID","VariableID","measuredat","value"]].groupby(["ICUSessionID","VariableID"]).apply(lambda x:linregress(x.measuredat,x.value)[0]).reset_index()
labdata_df_patients_slope.columns = [str(col) for col in labdata_df_patients_slope.columns.values]
labdata_df_patients_slope = labdata_df_patients_slope.rename(columns={'0':"slope"})
labdata_df_patients_slope.VariableID = labdata_df_patients_slope.VariableID.astype(str)
labdata_df_patients_slope = labdata_df_patients_slope.pivot(index='ICUSessionID', columns='VariableID')
labdata_df_patients_slope.columns = ['_'.join(col).rstrip('_') for col in labdata_df_patients_slope.columns.values]
labdata_df_patients_slope = labdata_df_patients_slope.reset_index()

labdata_df_patients_total = labdata_df_patients_agg.merge(labdata_df_patients_slope,how='left',on='ICUSessionID')
labdata_df_patients = None #Save RAM
labdata_df_patients_slope = None
labdata_df_patients_agg = None

C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\_stats_mstats_common.py:181: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\_stats_mstats_common.py:195: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\_stats_mstats_common.py:198: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


## diagnosisprocedures

In [153]:
diagnosis_and_procedures_df = pd.read_csv(data_path+"diagnosis_and_procedures.csv",sep=";", encoding = "ISO-8859-1")
diagnosis_and_procedures_df = diagnosis_and_procedures_df.rename(columns={"start_mins_since_icu_adm":"start_measuredat",
                                                                          "end_mins_since_icu_adm":"end_measuredat"})
diagnosis_and_procedures_df["value"]=1
diagnosis_and_procedures_df


,DiagnosisProceduresID,ICUSessionID,VariableID,Var_Abbr,Category,start_measuredat,end_measuredat,PlaceName,start_admissionday7_7,end_admissionday7_7,value
0,219,2,11596,ETT,Catheters and tubes,11,1118.0,Mouth,0,1.0,1
1,220,2,11597,CVC,Catheters and tubes,11,1118.0,Vena femoralis L,0,1.0,1
2,221,2,11598,ARTCATH,Catheters and tubes,14,1118.0,Arteria femoralis L,0,1.0,1
3,225,3,11597,CVC,Catheters and tubes,72,1608.0,Vena jugularis R,0,1.0,1
4,226,3,11598,ARTCATH,Catheters and tubes,72,1608.0,Arteria radialis R,0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
64080,155955,34293,11598,ARTCATH,Catheters and tubes,14,NaN,Arteria brachialis R,0,NaN,1
64081,155956,34293,11597,CVC,Catheters and tubes,1495,NaN,Vena jugularis R,1,NaN,1
64082,155950,34294,11598,ARTCATH,Catheters and tubes,18,NaN,Arteria radialis R,0,NaN,1
64083,155952,34294,11597,CVC,Catheters and tubes,18,589.0,Vena jugularis R,0,0.0,1


In [154]:
#NUMERIC ITEMS PREPROCESSING, this takes around 2 minutes
if non_biased_model:
    diagnosis_and_procedures_df_patients = (diagnosis_and_procedures_df[diagnosis_and_procedures_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='ICUSessionID')
else:
    diagnosis_and_procedures_df_patients = (diagnosis_and_procedures_df[diagnosis_and_procedures_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat"]],how='left',on='ICUSessionID')
    
diagnosis_and_procedures_df_patients["start_time_to_AF"]=(diagnosis_and_procedures_df_patients.AF_measuredat.values-margin_time) - diagnosis_and_procedures_df_patients.start_measuredat.values #add one margin_time to AF extra
diagnosis_and_procedures_df_patients["stop_time_to_AF"]=(diagnosis_and_procedures_df_patients.AF_measuredat.values-margin_time) - diagnosis_and_procedures_df_patients.end_measuredat.values #add one margin_time to AF extra

diagnosis_and_procedures_df_patients = diagnosis_and_procedures_df_patients[((diagnosis_and_procedures_df_patients.start_time_to_AF > (time_shift-margin_time)) & (diagnosis_and_procedures_df_patients.start_time_to_AF <= (time_shift+hours_to_first_AF-margin_time) ))|
                            ((diagnosis_and_procedures_df_patients.stop_time_to_AF > (time_shift-margin_time)) & (diagnosis_and_procedures_df_patients.stop_time_to_AF <= (time_shift+hours_to_first_AF-margin_time) ))]

for VariableID_loop in diagnosis_and_procedures_df.VariableID.unique():
    diagnosis_and_procedures_df_patients.VariableID = diagnosis_and_procedures_df_patients.VariableID.replace(VariableID_loop,diagnosis_and_procedures_df[diagnosis_and_procedures_df.VariableID==VariableID_loop].Var_Abbr.values[0])

diagnosis_and_procedures_df = None #RAM Optimization

diagnosis_and_procedures_df_patients_agg = diagnosis_and_procedures_df_patients[["ICUSessionID","VariableID","value"]].groupby(["ICUSessionID","VariableID"]).agg({'mean'}).reset_index()
diagnosis_and_procedures_df_patients_agg.VariableID = diagnosis_and_procedures_df_patients_agg.VariableID.astype(str)
diagnosis_and_procedures_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in diagnosis_and_procedures_df_patients_agg.columns.values]
diagnosis_and_procedures_df_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in diagnosis_and_procedures_df_patients_agg.columns.values]
diagnosis_and_procedures_df_patients_agg = diagnosis_and_procedures_df_patients_agg.pivot(index='ICUSessionID', columns='VariableID')
diagnosis_and_procedures_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in diagnosis_and_procedures_df_patients_agg.columns.values]

diagnosis_and_procedures_df_patients_total = diagnosis_and_procedures_df_patients_agg
diagnosis_and_procedures_df_patients = None #Save RAM
diagnosis_and_procedures_df_patients_slope = None
diagnosis_and_procedures_df_patients_agg = None

## medtreatment

In [155]:
medtreatment_df = pd.read_csv(data_path+"medtreatment.csv",sep=";", encoding = "ISO-8859-1")
medtreatment_df = medtreatment_df.rename(columns={"start_mins_since_icu_adm":"start_measuredat","Dose":"value",
                                                                          "end_mins_since_icu_adm":"end_measuredat"})
medtreatment_df = medtreatment_df[~medtreatment_df.value.str.contains(",")]
medtreatment_df["value"] = pd.to_numeric(medtreatment_df["value"])
medtreatment_df

,MedTreatmentID,ICUSessionID,VariableID,Var_Abbr,Category,LocalID,start_measuredat,end_measuredat,Route,RouteGroup,value,Unit,Maincomponent,start_admissionday7_7,end_admissionday7_7,ATCCode
0,327102,6212,11487,NS_propofol,Pharmaceuticals,1,1396,1397,IVV,IV,5.44444,mg,True,1,1,N01AX10
1,327104,6212,11487,NS_propofol,Pharmaceuticals,1,1397,1420,IVV,IV,57.77556,mg,True,1,1,N01AX10
2,327106,6212,11487,NS_propofol,Pharmaceuticals,1,11737,11871,IVV,IV,403.85000,mg,True,8,8,N01AX10
3,327108,6212,11487,NS_propofol,Pharmaceuticals,1,11871,11886,IVV,IV,48.88889,mg,True,8,8,N01AX10
4,327110,6212,11487,NS_propofol,Pharmaceuticals,1,11886,11886,IVV,IV,0.55000,mg,True,8,8,N01AX10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786266,5340056,33182,11436,CV_norepinephrine,Pharmaceuticals,1,1301,1301,CVV,IV,0.01116,mg,True,1,1,C01CA03
786267,5340057,33182,11436,CV_norepinephrine,Pharmaceuticals,1,1301,1302,CVV,IV,0.02681,mg,True,1,1,C01CA03
786268,5340058,33182,11436,CV_norepinephrine,Pharmaceuticals,1,1302,1304,CVV,IV,0.01335,mg,True,1,1,C01CA03
786269,5340059,33182,11436,CV_norepinephrine,Pharmaceuticals,1,1304,1304,CVV,IV,0.00123,mg,True,1,1,C01CA03


In [156]:
#NUMERIC ITEMS PREPROCESSING, this takes around 2 minutes
if non_biased_model:
    medtreatment_df_patients = (medtreatment_df[medtreatment_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='ICUSessionID')
else:
    medtreatment_df_patients = (medtreatment_df[medtreatment_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat"]],how='left',on='ICUSessionID')
    
medtreatment_df_patients["start_time_to_AF"]=(medtreatment_df_patients.AF_measuredat.values-margin_time) - medtreatment_df_patients.start_measuredat.values #add one margin_time to AF extra
medtreatment_df_patients["stop_time_to_AF"]=(medtreatment_df_patients.AF_measuredat.values-margin_time) - medtreatment_df_patients.end_measuredat.values #add one margin_time to AF extra

medtreatment_df_patients = medtreatment_df_patients[((medtreatment_df_patients.start_time_to_AF > (time_shift-margin_time)) & (medtreatment_df_patients.start_time_to_AF <= (time_shift+hours_to_first_AF-margin_time) ))|
                            ((medtreatment_df_patients.stop_time_to_AF > (time_shift-margin_time)) & (medtreatment_df_patients.stop_time_to_AF <= (time_shift+hours_to_first_AF-margin_time) ))]

for VariableID_loop in medtreatment_df.VariableID.unique():
    medtreatment_df_patients.VariableID = medtreatment_df_patients.VariableID.replace(VariableID_loop,medtreatment_df[medtreatment_df.VariableID==VariableID_loop].Var_Abbr.values[0])

medtreatment_df = None #RAM Optimization

medtreatment_df_patients_agg = medtreatment_df_patients[["ICUSessionID","VariableID","value"]].groupby(["ICUSessionID","VariableID"]).agg({'mean'}).reset_index()
medtreatment_df_patients_agg.VariableID = medtreatment_df_patients_agg.VariableID.astype(str)
medtreatment_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in medtreatment_df_patients_agg.columns.values]
medtreatment_df_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in medtreatment_df_patients_agg.columns.values]
medtreatment_df_patients_agg = medtreatment_df_patients_agg.pivot(index='ICUSessionID', columns='VariableID')
medtreatment_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in medtreatment_df_patients_agg.columns.values]

medtreatment_df_patients_total = medtreatment_df_patients_agg
medtreatment_df_patients = None #Save RAM
medtreatment_df_patients_slope = None
medtreatment_df_patients_agg = None

## izis dervals

In [157]:
izisdervals_df = pd.read_csv(data_path+"izisdervals.csv",sep=";", encoding = "ISO-8859-1")
izisdervals_df.varvalue = pd.to_numeric(izisdervals_df.varvalue)
izisdervals_df = izisdervals_df.rename(columns={"icusessionid":"ICUSessionID","mins_since_admission":"measuredat","varvalue":"value","variablename":"Var_Abbr"})
izisdervals_df

,ICUSessionID,measuredat,Var_Abbr,value
0,2,-748,BALANS-Tot.,0.000000
1,2,-28,BALANS-Tot.,0.000000
2,2,32,BALANS-Tot.,-130.000000
3,2,51,BALANS-Tot.,-127.762117
4,2,52,BALANS-Tot.,-124.287117
...,...,...,...,...
17411212,34294,1193,BALANS-Tot.,571.681544
17411213,34294,1204,BALANS-Tot.,575.696544
17411214,34294,1212,BALANS-Tot.,594.061810
17411215,34294,1273,BALANS-Tot.,755.394810


In [158]:
#NUMERIC ITEMS PREPROCESSING, this takes around 2 minutes
if non_biased_model:
    izisdervals_df_patients = (izisdervals_df[izisdervals_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='ICUSessionID')
else:
    izisdervals_df_patients = (izisdervals_df[izisdervals_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat"]],how='left',on='ICUSessionID')

izisdervals_df_patients["time_to_AF"]=(izisdervals_df_patients.AF_measuredat.values-margin_time) - izisdervals_df_patients.measuredat.values #add one margin_time to AF extra
izisdervals_df_patients = izisdervals_df_patients[(izisdervals_df_patients.time_to_AF > (time_shift-margin_time)) & (izisdervals_df_patients.time_to_AF <= (time_shift+hours_to_first_AF-margin_time) )]

for VariableID_loop in izisdervals_df.Var_Abbr.unique():
    izisdervals_df_patients.Var_Abbr = izisdervals_df_patients.Var_Abbr.replace(VariableID_loop,izisdervals_df[izisdervals_df.Var_Abbr==VariableID_loop].Var_Abbr.values[0])

izisdervals_df = None #RAM Optimization

izisdervals_df_patients_agg = izisdervals_df_patients[["ICUSessionID","Var_Abbr","value"]].groupby(["ICUSessionID","Var_Abbr"]).agg({'mean','min','max',pd.DataFrame.kurt}).reset_index()
izisdervals_df_patients_agg.Var_Abbr = izisdervals_df_patients_agg.Var_Abbr.astype(str)
izisdervals_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in izisdervals_df_patients_agg.columns.values]
izisdervals_df_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in izisdervals_df_patients_agg.columns.values]
izisdervals_df_patients_agg = izisdervals_df_patients_agg.pivot(index='ICUSessionID', columns='Var_Abbr')
izisdervals_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in izisdervals_df_patients_agg.columns.values]

izisdervals_df_patients_total = izisdervals_df_patients_agg
izisdervals_df_patients = None #Save RAM
izisdervals_df_patients_slope = None
izisdervals_df_patients_agg = None

## izis monvals

In [159]:
izismonvals_df = pd.read_csv(data_path+"izismonvals.csv",sep=";", encoding = "ISO-8859-1")
izismonvals_df = izismonvals_df[~izismonvals_df.varvalue.str.contains(",")]
izismonvals_df.varvalue = pd.to_numeric(izismonvals_df.varvalue)
izismonvals_df = izismonvals_df.rename(columns={"icusessionid":"ICUSessionID","mins_since_admission":"measuredat","varvalue":"value","variablename":"Var_Abbr"})
izismonvals_df

,ICUSessionID,measuredat,Var_Abbr,value
0,3,312,CVD,16.0
1,3,372,CVD,16.0
2,3,432,CVD,7.0
3,3,492,CVD,10.0
4,3,552,CVD,6.0
...,...,...,...,...
1169540,34294,942,CVD,13.0
1169541,34294,972,CVD,13.0
1169542,34294,1032,CVD,11.0
1169543,34294,1092,CVD,14.0


In [160]:
#NUMERIC ITEMS PREPROCESSING, this takes around 2 minutes
if non_biased_model:
    izismonvals_df_patients = (izismonvals_df[izismonvals_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='ICUSessionID')
else:
    izismonvals_df_patients = (izismonvals_df[izismonvals_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat"]],how='left',on='ICUSessionID')

izismonvals_df_patients["time_to_AF"]=(izismonvals_df_patients.AF_measuredat.values-margin_time) - izismonvals_df_patients.measuredat.values #add one margin_time to AF extra
izismonvals_df_patients = izismonvals_df_patients[(izismonvals_df_patients.time_to_AF > (time_shift-margin_time)) & (izismonvals_df_patients.time_to_AF <= (time_shift+hours_to_first_AF-margin_time) )]

for VariableID_loop in izismonvals_df.Var_Abbr.unique():
    izismonvals_df_patients.Var_Abbr = izismonvals_df_patients.Var_Abbr.replace(VariableID_loop,izismonvals_df[izismonvals_df.Var_Abbr==VariableID_loop].Var_Abbr.values[0])

izismonvals_df = None #RAM Optimization

izismonvals_df_patients_agg = izismonvals_df_patients[["ICUSessionID","Var_Abbr","value"]].groupby(["ICUSessionID","Var_Abbr"]).agg({'mean','min','max',pd.DataFrame.kurt}).reset_index()
izismonvals_df_patients_agg.Var_Abbr = izismonvals_df_patients_agg.Var_Abbr.astype(str)
izismonvals_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in izismonvals_df_patients_agg.columns.values]
izismonvals_df_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in izismonvals_df_patients_agg.columns.values]
izismonvals_df_patients_agg = izismonvals_df_patients_agg.pivot(index='ICUSessionID', columns='Var_Abbr')
izismonvals_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in izismonvals_df_patients_agg.columns.values]

izismonvals_df_patients_total = izismonvals_df_patients_agg
izismonvals_df_patients = None #Save RAM
izismonvals_df_patients_slope = None
izismonvals_df_patients_agg = None

## mondata val

In [161]:
mondata_validated_df = pd.read_csv(data_path+"mondata_validated.csv",sep=";", encoding = "ISO-8859-1")
mondata_validated_df = mondata_validated_df.rename(columns={"mins_since_icu_adm":"measuredat","VariableValue":"value"})

In [162]:
mondata_validated_df.Var_Abbr.unique()

array(['IBP_D', 'HR_ALL', 'IBP_M', 'RESPRATE_ALL', 'FIO2_MV_ALL', 'IBP_S',
       'URINE_VOL', 'SAO2_ALL', 'VENT_MODE', 'PEEP_M_ALL', 'MV', 'NIBPS',
       'NIBPM', 'NIBPD', 'PF RATIO', 'NO', 'URINE_VOL24'], dtype=object)

In [163]:
#NUMERIC ITEMS PREPROCESSING
if non_biased_model:
    mondata_validated_df_patients = (mondata_validated_df[mondata_validated_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='ICUSessionID')
else:
    mondata_validated_df_patients = (mondata_validated_df[mondata_validated_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat"]],how='left',on='ICUSessionID')

mondata_validated_df_patients["time_to_AF"]=(mondata_validated_df_patients.AF_measuredat.values-margin_time) - mondata_validated_df_patients.measuredat.values #add one margin_time to AF extra
mondata_validated_df_patients = mondata_validated_df_patients[(mondata_validated_df_patients.time_to_AF > (time_shift-margin_time)) & (mondata_validated_df_patients.time_to_AF <= (time_shift+hours_to_first_AF-margin_time) )]

for VariableID_loop in mondata_validated_df.VariableID.unique():
    mondata_validated_df_patients.VariableID = mondata_validated_df_patients.VariableID.replace(VariableID_loop,mondata_validated_df[mondata_validated_df.VariableID==VariableID_loop].Var_Abbr.values[0])

mondata_validated_df = None #RAM Optimization

mondata_validated_df_patients_agg = mondata_validated_df_patients[["ICUSessionID","VariableID","value"]].groupby(["ICUSessionID","VariableID"]).agg({'mean','min','max',pd.DataFrame.kurt}).reset_index()
mondata_validated_df_patients_agg.VariableID = mondata_validated_df_patients_agg.VariableID.astype(str)
mondata_validated_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in mondata_validated_df_patients_agg.columns.values]
mondata_validated_df_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in mondata_validated_df_patients_agg.columns.values]
mondata_validated_df_patients_agg = mondata_validated_df_patients_agg.pivot(index='ICUSessionID', columns='VariableID')
mondata_validated_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in mondata_validated_df_patients_agg.columns.values]

def linreg_except(x,value,measuredat):
    try:
        return linregress(x[measuredat],x[value])[0]
    except:
        return np.nan

mondata_validated_df_patients_slope = mondata_validated_df_patients[["ICUSessionID","VariableID","measuredat","value"]].groupby(["ICUSessionID","VariableID"]).apply(lambda x:linreg_except(x,"value","measuredat")).reset_index()
mondata_validated_df_patients_slope.columns = [str(col) for col in mondata_validated_df_patients_slope.columns.values]
mondata_validated_df_patients_slope = mondata_validated_df_patients_slope.rename(columns={'0':"slope"})
mondata_validated_df_patients_slope.VariableID = mondata_validated_df_patients_slope.VariableID.astype(str)
mondata_validated_df_patients_slope = mondata_validated_df_patients_slope.pivot(index='ICUSessionID', columns='VariableID')
mondata_validated_df_patients_slope.columns = ['_'.join(col).rstrip('_') for col in mondata_validated_df_patients_slope.columns.values]
mondata_validated_df_patients_slope = mondata_validated_df_patients_slope.reset_index()

mondata_validated_df_patients_total = mondata_validated_df_patients_agg.merge(mondata_validated_df_patients_slope,how='left',on='ICUSessionID')
mondata_validated_df_patients = None #Save RAM
mondata_validated_df_patients_slope = None
mondata_validated_df_patients_agg = None

C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\_stats_mstats_common.py:181: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\_stats_mstats_common.py:195: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\_stats_mstats_common.py:198: RuntimeWarning: invalid value encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)


## scores

In [164]:
sofa_df = pd.read_csv(data_path+"sofa_recalc.csv",sep=";", encoding = "ISO-8859-1")
sofa_df = sofa_df.fillna(0)
sofa_df = sofa_df.rename(columns={"icusessionid":"ICUSessionID"})
sofa_df["sofa"] = sofa_df.resp+sofa_df.coag+sofa_df.liver+sofa_df.cardio+sofa_df.cns+sofa_df.renal
sofa_df

,ICUSessionID,AdmissionDay,resp,coag,liver,cardio,cns,renal,sofa
0,2,0,4.0,3.0,2.0,4.0,0.0,2.0,15.0
1,2,1,4.0,4.0,2.0,4.0,0.0,4.0,18.0
2,3,0,2.0,0.0,0.0,1.0,0.0,0.0,3.0
3,3,1,0.0,0.0,0.0,0.0,0.0,3.0,3.0
4,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
136091,34293,2,2.0,0.0,0.0,4.0,1.0,0.0,7.0
136092,34293,3,1.0,0.0,0.0,1.0,1.0,0.0,3.0
136093,34293,4,2.0,0.0,0.0,1.0,0.0,0.0,3.0
136094,34294,0,0.0,2.0,2.0,4.0,4.0,2.0,14.0


In [165]:
#NUMERIC ITEMS PREPROCESSING
if non_biased_model:
    sofa_df_patients = (sofa_df[sofa_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat","date_corresponds_to_AF_admid","ICUAdmissionTime"]],how='left',on='ICUSessionID')
else:
    sofa_df_patients = (sofa_df[sofa_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat","ICUAdmissionTime"]],how='left',on='ICUSessionID')

sofa_df_patients["AF_admissionday"] = ((sofa_df_patients.ICUAdmissionTime+pd.to_timedelta(sofa_df_patients.AF_measuredat,"m")).dt.date - sofa_df_patients.ICUAdmissionTime.dt.date).dt.days
sofa_df_total = sofa_df_patients[sofa_df_patients.AF_admissionday == sofa_df_patients.AdmissionDay][["ICUSessionID","sofa"]]

## Merging them all

In [166]:
AF_full_dataset = AF_admission_dataset.merge(sofa_df_total,how='left',on='ICUSessionID').merge(
    labdata_df_patients_total,how='left',on='ICUSessionID').merge(
    mondata_validated_df_patients_total,how='left',on='ICUSessionID').merge(
    izismonvals_df_patients_total,how='left',on='ICUSessionID').merge(
    izisdervals_df_patients_total,how='left',on='ICUSessionID').merge(
    medtreatment_df_patients_total,how='left',on='ICUSessionID').merge(
    diagnosis_and_procedures_df_patients_total,how='left',on='ICUSessionID')

In [167]:
AF_full_dataset

,ICUSessionID,HospAdmissionID,ICUAdmissionSequence,AgeOnAdmission,ICUAdmissionTime,icu_los,icuurg,Length,Weight,bmi,...,mean_VI_calcium carbonate,mean_VI_calcium chloride,mean_ARTCATH,mean_CVC,mean_DIALYS_CVVH,mean_DIALYS_CVVHD,mean_DIALYS_IHD,mean_DIALYS_SLEDD,mean_DIAL_CATH,mean_ETT
0,17985,16514,1,21.31,2017-05-31 18:53:00,2800,Elective,169.0,60,21.00767,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22136,20268,1,29.37,2018-09-18 00:56:00,17038,Elective,165.0,65,23.87511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,20510,18808,1,49.72,2018-03-05 16:53:00,5731,Elective,165.0,80,29.38476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17798,16345,1,47.69,2017-05-10 23:41:00,2429,Elective,186.0,72,20.81165,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12642,11612,1,75.13,2016-04-18 00:03:00,2498,Elective,170.0,70,24.22145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13654,4825,4881,1,77.91,2014-06-16 18:02:00,2758,Elective,165.0,55,20.20202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13655,20911,19167,1,35.52,2018-04-18 23:32:00,6802,Elective,175.0,63,20.57143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13656,7735,7088,1,71.81,2015-07-05 14:35:00,2629,Elective,160.0,85,33.20312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13657,4053,4250,1,48.92,2014-03-18 15:59:00,21311,Elective,170.0,90,31.14187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
data_save_path = "./Data/312_Validatie_VKF_Predictiemodellen/"

if non_biased_model:
    if not six_hour_model:
        if one_half_hour_model:
            AF_full_dataset.to_csv(data_save_path+"AF_dataset_1_5_hours_no_af_distribution_matched.csv",index=False)
        else:
            AF_full_dataset.to_csv(data_save_path+"AF_dataset_12_hours_no_af_distribution_matched.csv",index=False)
    else:
        AF_full_dataset.to_csv(data_save_path+"AF_dataset_"+str(int(time_shift/60))+"_hours_no_af_distribution_matched.csv",index=False)
else:
    if not six_hour_model:
        if one_half_hour_model:
            AF_full_dataset.to_csv(data_save_path+"AF_dataset_1_5_hours.csv",index=False)
        else:
            AF_full_dataset.to_csv(data_save_path+"AF_dataset_12_hours.csv",index=False)
    else:
        AF_full_dataset.to_csv(data_save_path+"AF_dataset_"+str(int(time_shift/60))+"_hours.csv",index=False)

### Causal feature extraction

In [27]:
data_save_path = "./Data/312_Validatie_VKF_Predictiemodellen/"

In [33]:
if one_half_hour_model:
    if non_biased_model:
        AF_dataset = pd.read_csv(data_save_path+"AF_dataset_1_5_hours_no_af_distribution_matched.csv") 
    else:
        AF_dataset = pd.read_csv(data_save_path+"AF_dataset_1_5_hours.csv")
else:

    if non_biased_model:
        AF_dataset = pd.read_csv(data_save_path+"AF_dataset_12_hours_no_af_distribution_matched.csv") 
    else:
        AF_dataset = pd.read_csv(data_save_path+"AF_dataset_12_hours.csv")

if six_hour_model:
    if non_biased_model:
        AF_dataset = pd.read_csv(data_save_path+"AF_dataset_6_hours_no_af_distribution_matched.csv") 
    else:
        AF_dataset = pd.read_csv(data_save_path+"AF_dataset_6_hours.csv")
        
AF_dataset = AF_dataset.rename(columns={"HospAdmissionID":"patientid","AgeOnAdmission":"Age"})
AF_dataset.loc[AF_dataset.sepsis_extra_opn_measuredat > AF_dataset.AF_measuredat,"sepsis_bool_extra_opn"]=0

AF_dataset.loc[(~AF_dataset.bmi.isna())&(AF_dataset.bmi.str.contains(",")),'bmi']=np.NaN

AF_admission_dataset = AF_dataset

In [34]:
causal = "NOR" #"PEEP" or "NOR"

if causal == "NOR":
    causal_time_df = pd.read_csv(data_path+"medtreatment.csv",sep=";", encoding = "ISO-8859-1")
    causal_time_df = causal_time_df.rename(columns={"start_mins_since_icu_adm":"start_measuredat","Dose":"value",
                                                                              "end_mins_since_icu_adm":"end_measuredat"})
    causal_time_df = causal_time_df[~causal_time_df.value.str.contains(",")]
    causal_time_df["value"] = pd.to_numeric(causal_time_df["value"])
    causal_time_df = causal_time_df[causal_time_df.Var_Abbr == "CV_norepinephrine"]

elif causal =="PEEP":
    
    causal_time_df = pd.read_csv(data_path+"mondata_validated.csv",sep=";", encoding = "ISO-8859-1")
    causal_time_df = causal_time_df.rename(columns={"mins_since_icu_adm":"measuredat","VariableValue":"value"})
    causal_time_df = causal_time_df[causal_time_df.Var_Abbr=="PEEP_M_ALL"]


In [35]:
# causal_time_df

In [36]:
from scipy.stats import linregress
#specify hours before AF. IF YOU WANT A 12 HOUR EXTRA WINDOW, specify margin_time as 0, if you want the 1.5 hour model, specify margin_time as 1.5 and this as margin time.
#Min of time_shift is equal to margin_time
#To run this code, make sure you have enough RAM, restart the whole notebook and run this

causal_time_window = 6*60 #in minutes

time_shift = causal_time_window
margin_time = 0

##CAUSAL PREPROCESSING for causal time window
if causal == "PEEP":
    
    #NUMERIC ITEMS PREPROCESSING, this takes around 2 minutes
    Causal_df_patients = (causal_time_df[causal_time_df.ICUSessionID.isin(AF_admission_dataset.ICUSessionID)]).merge(AF_admission_dataset[["ICUSessionID","AF_measuredat","date_corresponds_to_AF_admid"]],how='left',on='ICUSessionID')

    Causal_df_patients["time_to_AF"]=(Causal_df_patients.AF_measuredat.values-margin_time) - Causal_df_patients.measuredat.values #add one margin_time to AF extra
    Causal_df_patients = Causal_df_patients[(Causal_df_patients.time_to_AF > (margin_time)) & (Causal_df_patients.time_to_AF <= (causal_time_window-margin_time-1) )]

    Causal_df_patients_agg = Causal_df_patients[["ICUSessionID","Var_Abbr","value"]].groupby(["ICUSessionID","Var_Abbr"]).agg({'mean'}).reset_index()
    Causal_df_patients_agg.Var_Abbr = Causal_df_patients_agg.Var_Abbr.astype(str)
    Causal_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in Causal_df_patients_agg.columns.values]
    Causal_df_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in Causal_df_patients_agg.columns.values]
    Causal_df_patients_agg = Causal_df_patients_agg.pivot(index='ICUSessionID', columns='Var_Abbr')
    Causal_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in Causal_df_patients_agg.columns.values]    
    
    Causal_df_patients_agg = Causal_df_patients_agg.rename(columns={"mean_PEEP_M_ALL":"causal_PEEP_M_ALL"})
    
    causal_time_df = None #RAM Optimization

elif causal == "NOR":
    
    Causal_df_patients = causal_time_df.merge(AF_admission_dataset,how="left",on="ICUSessionID")
    

    Causal_df_patients["start_time_to_AF"]=(Causal_df_patients.AF_measuredat.values-margin_time) - Causal_df_patients.start_measuredat.values #add one margin_time to AF extra
    Causal_df_patients["stop_time_to_AF"]=(Causal_df_patients.AF_measuredat.values-margin_time) - Causal_df_patients.end_measuredat.values #add one margin_time to AF extra

    Causal_df_patients = Causal_df_patients[((Causal_df_patients.start_time_to_AF > (time_shift-margin_time)) & (Causal_df_patients.start_time_to_AF <= (time_shift+hours_to_first_AF-margin_time) ))|
                                ((Causal_df_patients.stop_time_to_AF > (time_shift-margin_time)) & (Causal_df_patients.stop_time_to_AF <= (time_shift+hours_to_first_AF-margin_time) ))]

    Causal_df_patients_agg = Causal_df_patients[["ICUSessionID","Var_Abbr","value"]].groupby(["ICUSessionID","Var_Abbr"]).agg({'mean'}).reset_index()

    Causal_df_patients_agg.Var_Abbr = Causal_df_patients_agg.Var_Abbr.astype(str)
    Causal_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in Causal_df_patients_agg.columns.values]
    
    Causal_df_patients_agg.columns = [col.replace('value_','') if 'value_' in col else col for col in Causal_df_patients_agg.columns.values]

    Causal_df_patients_agg = Causal_df_patients_agg.pivot(index='ICUSessionID', columns='Var_Abbr')
    Causal_df_patients_agg.columns = ['_'.join(col).rstrip('_') for col in Causal_df_patients_agg.columns.values]
    
    Causal_df_patients_agg = Causal_df_patients_agg.rename(columns={"mean_CV_norepinephrine":"causal_CV_norepinephrine"})
    
    causal_time_df = None #RAM Optimization

AF_dataset = AF_admission_dataset.merge(Causal_df_patients_agg,how='left',on='ICUSessionID')

In [37]:
if non_biased_model:
    AF_dataset.to_csv(data_save_path+causal+"_causal_AF_dataset_causal_window_"+str(int(causal_time_window/60))+"_hours_no_af_distribution_matched.csv",index=False)
else:
    AF_dataset.to_csv(data_save_path+causal+"_causal_AF_dataset_causal_window_"+str(int(causal_time_window/60))+"_hours.csv",index=False)

: 